# Decison Tree Model by Tensorflow Decision Forests

## 1. Load data into python
Change user id

In [6]:
#Vega Setup
import logging
import pandas as pd
import numpy as np
from afterpay_gdp_interfaces import RedshiftHook

import datetime
import pytz
CST = pytz.timezone('Asia/Shanghai')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
vega = RedshiftHook(cluster='vega', okta_username='songli@squareup.com') # for vega connection
def vega_execute(query):
    import datetime
    """
        vega execute SQL wrapper to commit everytime
    """
    t_start = datetime.datetime.now()
    with vega.get_conn() as vega_conn:
        with vega_conn.cursor() as cur:
            cur.execute(query)
        vega_conn.commit()
        t_end = datetime.datetime.now()
        logging.info("Vega Query Finished. Time used: {}".format(str(t_end - t_start)))
        # vega_conn.close()

apply some initial filtering when pulling sample data

In [7]:
q="""
select
*
from sandbox_analytics_au.feature_base_sl drv
where
    par_region='AU'
    and lower(replace(in_flight_order_shipping_address_address_1,' ',''))!=lower(replace(consumer_address_1,' ',''))
    and lower(replace(in_flight_order_shipping_address_city,' ',''))!=lower(replace(consumer_city,' ',''))
    and in_flight_order_shipping_address_address_1!=''
    and days_since_first_order_date>42
    and ((TIMESTAMP 'epoch' + c_latest_login_2fa_success_timestamp::FLOAT *INTERVAL '1 second')>=cast(checkout_time as date)-30)
    and (case when nvl(in_flight_order_amount,'0')='' then '0' else in_flight_order_amount end)::float>200
;"""

In [23]:
rule_perf.head()

,order_token,checkpoint,checkout_time,consumer_id,par_process_date,par_region,merchant_id,consumer_state,in_flight_order_shipping_address_state,in_flight_order_shipping_address_postcode,consumer_contact_address_postcode,in_flight_order_shipping_address_name,in_flight_order_shipping_address_address_1,in_flight_order_shipping_address_address_2,consumer_name,in_flight_order_consumer_name,in_flight_order_consumer_email,consumer_email,in_flight_order_recipient_email,in_flight_order_recipient_message,in_flight_order_recipient_name,in_flight_order_shipping_address_mobile,bp_c_trusted_merch_side_email_yn,bp_c_trusted_shipping_address_hash,in_flight_order_shipping_address_hash,session_user_agent,consumer_is_first_order,region,region_code,c_latest_login_2fa_success_method,city,sp_payment_method_last_card_scan_is_successful,attempt_payment_method,consumer_mobile,consumer_email_domain,consumer_given_names,consumer_address_1,consumer_address_2,consumer_city,consumer_postcode,in_flight_order_shipping_address_city,session_client_ip,bp_m_mcc_code,bp_m_mcc_desc,in_flight_order_billing_address_name,in_flight_order_billing_address_address_1,in_flight_order_billing_address_address_2,in_flight_order_billing_address_city,in_flight_order_billing_address_state,in_flight_order_billing_address_postcode,...,sp_c_online_order_count_h1_0,sp_c_online_order_count_h24_0,sp_c_online_order_count_h6_0,sp_c_online_order_count_h168_0,days_since_first_order_date,device_age_in_days,bp_c_merch_side_email_age_days,whitepages_identity_check_score,model_gibberish_merchant_side_email_august_2022_score,model_gibberish_consumer_profile_email_august_2022_score,bp_entity_linking_hop0_cnt_by_merch_side_email_d7,bp_entity_linking_hop0_tot_order_cnt_by_merch_side_email_d7,bp_entity_linking_hop0_tot_order_amt_by_merch_side_email_d1,bp_entity_linking_hop0_tot_order_amt_by_merch_side_email_d3,sp_c_online_ordr_attmpt_credit_card_cnt_h12_0,sp_c_pymt_attmpt_cnt_h1_0,sp_c_failed_pymt_cnt_h12_0,sp_c_manual_pymt_attmpt_cnt_h12_0,pymt_method_1st_attempt_timestamp,ip_to_profile_address_postcode_geo_distance_miles,consumer_postcode_to_shipping_address_postcode_geo_distance_miles_v2,ip_to_shipping_address_postcode_geo_distance_miles,model_online_cb_global_july_2022_score,model_online_ato_global_july_2022_score,model_online_ato_global_september_2022_score,model_online_cb_all_global_september_2022_score,num_days_to_booking,in_flight_card_name_vs_profile_name,bp_c_latest_approved_credit_limit,bp_c_latest_approved_credit_limit_v2,c_latest_login_2fa_success_timestamp,c_latest_pwd_reset_success_timestamp,c_latest_sms_login_2fa_success_timestamp,c_latest_email_login_2fa_success_timestamp,c_latest_email_pwd_reset_success_timestamp,sp_c_latest_profile_address_change_timestamp,c_latest_sms_pwd_reset_success_timestamp,c_latest_login_2fa_required_timestamp,sp_c_latest_profile_email_change_timestamp,sp_c_latest_profile_phone_number_change_timestamp,rule_ids,control_group,gmv_local,gmv_aud,gwr_ind,ato_ind,sf_ind,loss_ind,order_date,order_week
0,001.2v4iq31rlnbkpt2kjtgu7doaqdlfinqv4ag58rufjm...,CHECKOUT_CONFIRM,2022-12-19 06:41:19,e642c493-4fcd-4ef3-9794-adab0290ab92,2022-12-19,AU,96902,VIC,VIC,3207,3910,Luke Wagner,202/320 plumber street,5-40 myrtle street langwarrin,Luke Wagner,Luke Wagner,wagnerluke97@gmail.com,wagnerluke97@gmail.com,,,,610409621077,True,,801fe273651aa43e3159ff756e6cf250,Mozilla/5.0 (iPhone; CPU iPhone OS 15_4_1 like...,,Victoria,VIC,sms,Melbourne,,CREDIT_CARD,61409621077,gmail.com,Luke,40 Myrtle St,,Langwarrin,3910,Port Melbourne,2001:8004:1270:4d9a:a836:3dca:47d:1472,7922,Theatrical Producers (except Motion Pictures) ...,Luke Wagner,202/320 plumber street,5-40 myrtle street langwarrin,Port Melbourne,VIC,3207,...,0.0,0.0,0.0,0.0,1240.265324,0.0,1198.0,380.0,5.750197,5.750197,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.652176e+12,24.517692,26.115785,2.824016,0.830742,0.110185,8.138270,19.366077,0.0,74.0,1700.0,2000.0,1.671432e+09,1.652175e+09,1.671432e+09,0.0,0.0,0.000000e+00,1.652175e+09,1.67

In [8]:
%%time
print(datetime.datetime.now(CST))
rule_perf=vega.get_pandas_df(q)

2023-02-14 10:35:23.850847+08:00
CPU times: user 11 s, sys: 1.77 s, total: 12.8 s
Wall time: 28.7 s


In [9]:
rule_perf['order_date']=pd.to_datetime(rule_perf['checkout_time']).dt.date
import datetime
rule_perf['order_week']=rule_perf['order_date']-rule_perf['order_date'].apply(lambda x: datetime.timedelta(pd.to_datetime(x).weekday()))   

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/arrays/datetimelike.py:1190: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  PerformanceWarning,


### check initial fraud rate by week

In [17]:
rule_perf.pivot_table(values='in_flight_order_amount',index='order_week', columns='loss_ind', aggfunc='count')

loss_ind,0.0,1.0
order_week,,
2022-12-12,17656.0,13.0
2022-12-19,25824.0,4.0
2022-12-26,27131.0,NaN
2023-01-02,19423.0,5.0
2023-01-09,18560.0,12.0
2023-01-16,19063.0,7.0
2023-01-23,15259.0,2.0
2023-01-30,18477.0,6.0
2023-02-06,17809.0,6.0


## 2. Import related packages
If you have not installed tensorflow use 
```shell
!pip install --user tensorflow  
!pip install --user tensorflow_decision_forests
```

In [11]:
import tensorflow_decision_forests as tfdf
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

## 3. Preprocessing
Apply some filtering to the sample data.  
Apply feature exclusion list as some features should not be used (or you don't want to use) in the model.

In [18]:
# Apply some filter criteria
dataset_df = rule_perf[(pd.to_datetime(rule_perf['order_date'])>='2022-11-24')#&(pd.to_datetime(rule_perf['order_date'])<'2022-12-12')\
                       &(rule_perf['rule_ids']=='["Approved"]')
                       &~(rule_perf['control_group'].str.lower().str.contains('control'))
                       
                       & (rule_perf['days_since_first_order_date']>300)
                      ].copy()
# fill missing value
dataset_df['loss_ind'].fillna(0,inplace=True)

label = "loss_ind"

classes = dataset_df[label].unique().tolist()
print(f"Label classes: {classes}")

#Exclude some features from being predictors 
dataset_df2=dataset_df[list(set(dataset_df.columns.to_list())-
                           set(['checkout_time','checkout_time_f','ato_ind','sf_id', 'gwr_ind','par_process_date'
                                ,'in_flight_order_shipping_address_hash','rule_ids','consumer_city'
                                ,'gmv_local','gmv_aud','order_week', 'sf_ind','consumer_given_names'
                                ,'c_latest_email_login_2fa_success_timestamp','session_user_agent'
                                ,'merchant_id', 'merchant_id_f','consumer_id','consumer_email','order_date', 'order_week','consumer_id_f', 'in_flight_order_consumer_email', 'consumer_mobile'
                                ,'in_flight_order_consumer_name','consumer_name','consumer_postcode','consumer_birth_date_epoch_millis','highest_delphi_score_last_12_hours'
                               ]+dataset_df.columns[dataset_df.columns.str.contains('time')].to_list()
                               +dataset_df.columns[(dataset_df.columns.str.contains('_address'))&~(dataset_df.columns.str.contains('geo'))].to_list()
                              ))].copy()
#Format conversion
dataset_df2[dataset_df2.dtypes[dataset_df2.dtypes=='bool'].index]=\
    dataset_df2[dataset_df2.dtypes[dataset_df2.dtypes=='bool'].index].astype(str)

Label classes: [0.0, 1.0]


## 4. Train Test split
Test ratio controls the test sample size.

In [19]:
def split_dataset(dataset, test_ratio=0.1):
    """Splits a panda dataframe in two."""
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

train_ds_pd, test_ds_pd = split_dataset(dataset_df2)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

168663 examples in training, 18648 examples for testing.


/home/jupyter-songli/.local/lib/python3.7/site-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


## 5. Build Model
Some key parameters: 
- min_examples: controls the minumum size of the node. You need to balance between overfitting and fraud rate. 
- max_depth: depth of the tree

In [ ]:
%%time
# build model
model = tfdf.keras.RandomForestModel(min_examples=30,max_depth=5,sampling_with_replacement=False,bootstrap_size_ratio=1.0,num_trees=3)

# train model
model.fit(x=train_ds, validation_data=test_ds)

## 6. Visualize the tree 

In [21]:
with open("plot.html", "w") as f:
    f.write(tfdf.model_plotter.plot_model(model, tree_idx=0, max_depth=8))

from IPython.display import IFrame
IFrame(src='./plot.html', width=2000, height=600)

In [22]:
inspector=model.make_inspector()
inspector.extract_tree(tree_idx=0)

Tree(NonLeafNode(condition=(sp_c_fraud_decline_attempt_h12_0 >= 0.5; miss=False), pos_child=NonLeafNode(condition=(bp_m_mcc_desc in ['Department Stores', 'Electronics Shops', 'Sports and Riding Apparel Shops', 'Mens and Womens Clothing Shops', 'Travel Agencies and Tour Operators']; miss=False), pos_child=NonLeafNode(condition=(model_online_ato_global_september_2022_score >= 40.66364288330078; miss=False), pos_child=NonLeafNode(condition=(model_online_cb_all_global_september_2022_score >= 666.6171875; miss=False), pos_child=LeafNode(value=ProbabilityValue([0.7297297297297297, 0.2702702702702703],n=37.0), idx=10), neg_child=LeafNode(value=ProbabilityValue([0.9888888888888889, 0.011111111111111112],n=90.0), idx=9), value=ProbabilityValue([0.9133858267716536, 0.08661417322834646],n=127.0)), neg_child=LeafNode(value=ProbabilityValue([1.0, 0.0],n=238.0), idx=8), value=ProbabilityValue([0.9698630136986301, 0.030136986301369864],n=365.0)), neg_child=LeafNode(value=ProbabilityValue([1.0, 0.0],n

### Selected desired(and reasonable) nodes and check performance!

In [11]:
dataset_df[
(    
(dataset_df['sp_c_fraud_decline_attempt_d7_0']>0)
&(dataset_df['model_online_cb_global_july_2022_score']>4.8)
&(dataset_df['bp_c_merch_side_email_age_days']<=5)
)
]\
.pivot_table(values='in_flight_order_amount',index='order_week', columns='loss_ind', aggfunc='count', margins=True)

loss_ind,0.0,1.0,All
order_week,,,
2022-11-21,18,9,27
2022-11-28,20,3,23
2022-12-05,15,4,19
All,53,16,69
